## テスト用のxmlフォーマットの力場を作るノートブック

- inputは元々のxmlファイル
- outputは変換後のxmlファイルと変更箇所のtxtファイル

In [114]:
import xml.etree.ElementTree as ET
import itertools
import random
import copy
import re
import os

In [133]:
#path
pdb_filepath = "./alanine-dipeptide-nowater.pdb"
ff_input_filepath  = "./GB99dms.xml"
ff_output_xml_filepath = "./output.xml"
ff_output_txt_filepath = "./output.txt"

epoch_n = 1 #今何epoch目かを表す変数

In [132]:
#parameter
maximum_update_ratio = 0.03 #最大でffのパラメータをどの程度変えるか

In [98]:
#atom_type
phi_atom_type = [['C'], ['N'], ['CT'], ['C']]
psi_atom_type = [['N'], ['CT'], ['C'], ['N']]

## input xml format

In [99]:
tree = copy.deepcopy(ET.parse(ff_input_filepath)) #xml formatのff
root = tree.getroot()

In [100]:
# XMLから'PeriodicTorsionForce'を検索
for torsion in root.iter('PeriodicTorsionForce'):

    #各要素の中から、phiとpsiに関するパラメータを検索
    for ff_local in torsion:
        #print(list(ff_local.attrib.values())[0:4]) [0:4]でスライスすることで、アトムタイプを取り出す
        #phiに関係する二面角のパラメータを取り出す
        if list(ff_local.attrib.values())[0:4] == list(itertools.chain.from_iterable(phi_atom_type)):
            phi_ff_element = ff_local #xml.etree.ElementTree.Element型でphiに関連する行を抜き出す
            
        #psiに関係する二面角のパラメータを取り出す
        if list(ff_local.attrib.values())[0:4] == list(itertools.chain.from_iterable(psi_atom_type)):        
            psi_ff_element = ff_local

In [101]:
type(phi_ff_element)

xml.etree.ElementTree.Element

In [102]:
#辞書型でパラメータの値を取り出す
phi_ff_param_original = copy.deepcopy(phi_ff_element.attrib)
psi_ff_param_original = copy.deepcopy(psi_ff_element.attrib)

In [103]:
print(type(phi_ff_param))
phi_ff_param

<class 'dict'>


{'type1': 'C',
 'type2': 'N',
 'type3': 'CT',
 'type4': 'C',
 'periodicity1': '1',
 'phase1': '0.0',
 'k1': '-0.1396954369058983',
 'periodicity2': '2',
 'phase2': '0.0',
 'k2': '1.4020721749174487',
 'periodicity3': '3',
 'phase3': '0.0',
 'k3': '2.2722793075263983',
 'periodicity4': '4',
 'phase4': '0.0',
 'k4': '0.33249158784820976',
 'periodicity5': '2',
 'phase5': '1.5707963267948966',
 'k5': '1.669819011164073'}

In [104]:
#create forcefield
phi_ff_param_updated = dict()
psi_ff_param_updated = dict()

for key, value in phi_ff_param_original.items():
    #keyで条件分岐
    #type
    if re.search(r'type.*', key):
        phi_ff_param_updated[key] = value
        continue
    #periodicity
    if re.search(r'periodicity.*', key):
        phi_ff_param_updated[key] = value
        continue
    #phase
    if re.search(r'phase.*', key):
        phi_ff_param_updated[key] = value
        continue
    #k
    if re.search(r'k.*', key):
        k = float(value)
        #[-maximum_update_ratio, maximum_update_ratio]の範囲の乱数
        rnd = random.uniform(-maximum_update_ratio, maximum_update_ratio)
        k = k * (1 + rnd)
        phi_ff_param_updated[key] = str(k)
        continue

for key, value in psi_ff_param_original.items():
    #keyで条件分岐
    #type
    if re.search(r'type.*', key):
        psi_ff_param_updated[key] = value
        continue
    #periodicity
    if re.search(r'periodicity.*', key):
        psi_ff_param_updated[key] = value
        continue
    #phase
    if re.search(r'phase.*', key):
        psi_ff_param_updated[key] = value
        continue
    #k
    if re.search(r'k.*', key):
        k = float(value)
        #[-maximum_update_ratio, maximum_update_ratio]の範囲の乱数
        rnd = random.uniform(-maximum_update_ratio, maximum_update_ratio) 
        k = k * (1 + rnd)
        psi_ff_param_updated[key] = str(k)
        continue

In [105]:
#create xml file
tree = copy.deepcopy(ET.parse(ff_input_filepath)) #xml formatのff
root = tree.getroot()

In [106]:
# XMLから'PeriodicTorsionForce'を検索
for torsion in root.iter('PeriodicTorsionForce'):

    #各要素の中から、phiとpsiに関するパラメータを検索
    for ff_local in torsion:
        #print(list(ff_local.attrib.values())[0:4]) [0:4]でスライスすることで、アトムタイプを取り出す
        #phiに関係する二面角のパラメータを取り出す
        if list(ff_local.attrib.values())[0:4] == list(itertools.chain.from_iterable(phi_atom_type)):
            ff_local.attrib = phi_ff_param_updated
            
        #psiに関係する二面角のパラメータを取り出す
        if list(ff_local.attrib.values())[0:4] == list(itertools.chain.from_iterable(psi_atom_type)):        
            ff_local.attrib = psi_ff_param_updated

#output
tree.write(ff_output_xml_filepath)

In [112]:
phi_ff_param_updated

{'type1': 'C',
 'type2': 'N',
 'type3': 'CT',
 'type4': 'C',
 'periodicity1': '1',
 'phase1': '0.0',
 'k1': '-0.14138935790247983',
 'periodicity2': '2',
 'phase2': '0.0',
 'k2': '1.3790236468189723',
 'periodicity3': '3',
 'phase3': '0.0',
 'k3': '2.22147001071864',
 'periodicity4': '4',
 'phase4': '0.0',
 'k4': '0.32333508420380236',
 'periodicity5': '2',
 'phase5': '1.5707963267948966',
 'k5': '1.6794469950133148'}

## .txtファイルへの書き込み

In [127]:
with open(ff_output_txt_filepath, 'w') as f:
    #phiに関係する情報
    f.write('PHI_PARAMETER\n')
    for key,value in phi_ff_param_updated.items():
    	f.write(f'{key}\t{value}\n')
    #psiに関係する情報
    f.write('PSI_PARAMETER\n')
    for key,value in psi_ff_param_updated.items():
    	f.write(f'{key}\t{value}\n')